In [4]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics
from read_tfrecord import *
from mtcnn_model import Pnet,cls_ohem,bbox_ohem
from tqdm import tqdm
import os
import numpy as np

In [5]:
data_path = "data/12/train_PNet_landmark.tfrecord_shuffle"

In [6]:
def load_pokemon(mode='train'):
    """ 加载pokemon数据集的工具！
    :param root:    数据集存储的目录
    :param mode:    mode:当前加载的数据是train,val,还是test
    :return:
    """
    # # 创建数字编码表,范围0-4;
    # name2label = {}  # "sq...":0   类别名:类标签;  字典 可以看一下目录,一共有5个文件夹,5个类别：0-4范围;
    # for name in sorted(os.listdir(os.path.join(root))):     # 列出所有目录;
    #     if not os.path.isdir(os.path.join(root, name)):
    #         continue
    #     # 给每个类别编码一个数字
    #     name2label[name] = len(name2label.keys())
 
    # 读取Label信息;保存索引文件images.csv
    # [file1,file2,], 对应的标签[3,1] 2个一一对应的list对象。
    # 根据目录,把每个照片的路径提取出来,以及每个照片路径所对应的类别都存储起来，存储到CSV文件中。
    size = 12
    images,labels,boxes = red_tf(data_path,size)
 
    # 图片切割成，训练70%，验证15%，测试15%。
    if mode == 'train':                                                     # 70% 训练集
        images = images[:int(0.7 * len(images))]
        labels = labels[:int(0.7 * len(labels))]
        boxes  = boxes[:int(0.7 * len(boxes))]
    elif mode == 'val':                                                     # 15% = 70%->85%  验证集
        images = images[int(0.7 * len(images)):int(0.85 * len(images))]
        labels = labels[int(0.7 * len(labels)):int(0.85 * len(labels))]
        boxes = boxes[int(0.7 * len(boxes)):int(0.85 * len(boxes))]
    else:                                                                   # 15% = 70%->85%  测试集
        images = images[int(0.85 * len(images)):]
        labels = labels[int(0.85 * len(labels)):]
        boxes = boxes[int(0.85 * len(boxes)):]
    ima = tf.data.Dataset.from_tensor_slices(images)
    lab = tf.data.Dataset.from_tensor_slices(labels)
    roi = tf.data.Dataset.from_tensor_slices(boxes)
    # ima,lab,roi = preprocess(ima,lab,roi)
    train_data = tf.data.Dataset.zip((ima, lab, roi)).shuffle(1000).batch(32)
    train_data = list(train_data.as_numpy_iterator())
    return train_data

In [10]:
for i,(img,lab,boxes) in enumerate(load_pokemon("train")):
    print(i)
    print(img)
    print(boxes)
    break

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
0
[[[[ 9.00470257e-01  7.88879871e-01  7.78909564e-01]
   [ 1.74341947e-02  1.65860786e-03  0.00000000e+00]
   [ 4.45009530e-01  4.01061982e-01  3.81063819e-01]
   ...
   [ 7.14567900e-01  5.37726879e-01  4.43208456e-01]
   [ 5.84436297e-01  4.58168507e-01  4.82645035e-01]
   [-1.12697437e-01 -1.12697437e-01 -1.12697437e-01]]

  [[ 7.14567900e-01  5.81127524e-01  5.46915293e-01]
   [ 3.60244215e-02  1.31991673e-02  0.00000000e+00]
   [ 6.58797264e-01  6.03625715e-01  5.83328485e-01]
   ...
   [ 2.40516990e-01  8.37754756e-02  0.00000000e+00]
   [ 2.12631613e-01  8.63638222e-02  9.93174016e-02]
   [ 2.86992580e-01  2.06816673e-01  2.55535245e-01]]

  [[ 2.67293081e-02  1.01918224e-02  0.00000000e+00]
   [ 8.44699502e-01  7.45529115e-01  6.88569903e-01]
   [ 7.51748323e-01  6.62605941e-01  6.53233707e-01]
   ...
   [ 6.39097691e-02  2

In [13]:
def train(eopch):
    model = Pnet()
    if os.path.exists("pnet.h5"):
        model.load_weights("pnet.h5")
    else:
        model.load_weights('./Weights/pnet_wight/pnet_30.ckpt')
 
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    off = 1000
    acc_meter = metrics.Accuracy()
    for epoch in tqdm(range(eopch)):
 
        for i,(img,lab,boxes) in enumerate(load_pokemon("train")):
 
 
            #img = image_color_distort(img)
            # 开一个gradient tape, 计算梯度
            with tf.GradientTape() as tape:
                cls_prob, bbox_pred = model(img)
                cls_prob = tf.squeeze(cls_prob,[1,2])
                bbox_pred = tf.squeeze(bbox_pred,[1,2])
 
                cls_loss = cls_ohem(cls_prob, lab)
                bbox_loss = bbox_ohem(bbox_pred, boxes,lab)
                # landmark_loss = landmark_loss_fn(landmark_pred, landmark_batch, label_batch)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('Training loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 6))
 
 
        for i, (v_img, v_lab1, boxes) in enumerate(load_pokemon("val")):
            v_img = image_color_distort(v_img)
            with tf.GradientTape() as tape:
                cls_prob, bbox_pred = model(v_img)
                cls_prob = tf.squeeze(cls_prob,[1,2])
                bbox_pred = tf.squeeze(bbox_pred,[1,2])
 
                cls_loss = cls_ohem(cls_prob, v_lab1)
                bbox_loss = bbox_ohem(bbox_pred, boxes,v_lab1)
                # landmark_loss = landmark_loss_fn(landmark_pred, landmark_batch, label_batch)
                # accuracy = cal_accuracy(cls_prob, label_batch)
 
 
                total_loss_value = cls_loss + 0.5 * bbox_loss
                grads = tape.gradient(total_loss_value, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
            if i % 200 == 0:
                print('val___ loss (for one batch) at step %s: %s' % (i, float(total_loss_value)))
                print('Seen so far: %s samples' % ((i + 1) * 6))
    model.save_weights('./Weights/pnet_wight/pnet_30.ckpt')

In [14]:
train(30)

  0%|          | 0/30 [00:00<?, ?it/s]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.8359413146972656
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.4155491590499878
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.27837345004081726
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.44817817211151123
Seen so far: 6 samples


  3%|▎         | 1/30 [00:54<26:26, 54.72s/it]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.4671699106693268
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.40162765979766846
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.3424588441848755
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.4209800660610199
Seen so far: 6 samples


  7%|▋         | 2/30 [01:53<26:44, 57.32s/it]

...

 93%|█████████▎| 28/30 [26:16<01:49, 54.93s/it]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.03416148200631142
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.03891266882419586
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.1922222077846527
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.2704184949398041
Seen so far: 6 samples


 97%|█████████▋| 29/30 [27:10<00:54, 54.51s/it]

<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
Training loss (for one batch) at step 0: 0.04365459084510803
Seen so far: 6 samples
Training loss (for one batch) at step 200: 0.18410339951515198
Seen so far: 1206 samples
Training loss (for one batch) at step 400: 0.06513691693544388
Seen so far: 2406 samples
<MapDataset shapes: {image/encoded: (), image/label: (), image/roi: (4,)}, types: {image/encoded: tf.string, image/label: tf.int64, image/roi: tf.float32}>
val___ loss (for one batch) at step 0: 0.11707638204097748
Seen so far: 6 samples


100%|██████████| 30/30 [28:04<00:00, 56.14s/it]
